In [ ]:
import numpy as np
import json
from pathlib import Path
import plotly.express as px

In [ ]:
example_camera_path_fname = Path('example_camera_path.json')
data = json.load(open(example_camera_path_fname))
print(data.keys())

In [ ]:
T_camera_paths = np.array([path['camera_to_world'] for path in data['camera_path']]).reshape(-1, 4, 4)
str2arr = lambda s: np.fromstring(s[1:-1], sep=',').reshape(4, 4).T
T_keyframes = np.array([str2arr(kf['matrix']) for kf in data['keyframes']]).reshape(-1, 4, 4)
# print(T_camera_paths)
# print(T_keyframes)
print(T_camera_paths.shape, T_keyframes.shape)

Notice that 96 is the number of total frames: 4s * 24fps = 96 frames...

In [ ]:
fig = px.scatter_3d(x=T_camera_paths[:, 0, 3], y=T_camera_paths[:, 1, 3], z=T_camera_paths[:, 2, 3])
fig.add_scatter3d(x=T_keyframes[:, 0, 3], y=T_keyframes[:, 1, 3], z=T_keyframes[:, 2, 3])
fig.update_layout(scene_aspectmode='data')
fig.show()

Let's create just a perfect circle of camera frames

In [ ]:
# print(T_keyframes[:, :3, 3])
print(f'mean z height: {T_keyframes[:, 2, 3].mean()}')
print(f'mean radius: {np.sqrt((T_keyframes[:, 0, 3]**2 + T_keyframes[:, 1, 3]**2).mean())}')
fig = px.line(x=T_keyframes[:, 0, 3], y=T_keyframes[:, 1, 3])
fig.update_layout(scene=dict(aspectmode='data'), width=300, height=300)
fig.show()

In [ ]:
look_at = np.array([0, 0, 0])
R = 2.75
height = 1.0
data['fps'] = 3
num_images = data['fps'] * data['seconds']
t = np.linspace(0, np.pi*2, num_images, endpoint=False)[:, None]
positions = np.hstack((R * np.cos(t), R * np.sin(t), height*np.ones_like(t)))

def pos_lookat_to_matrix(pos, look_at):
    z = -(look_at - pos) / np.linalg.norm(look_at - pos)
    x = np.cross(np.array([0, 0, 1]), z)
    x = x / np.linalg.norm(x)
    y = np.cross(z, x)
    y = y / np.linalg.norm(y)
    return np.vstack((np.array([x, y, z, pos]).T, np.array([0, 0, 0, 1])))

T_camera_paths_new = np.array([pos_lookat_to_matrix(pos, look_at) for pos in positions])
fig = px.scatter_3d(x=T_camera_paths[:, 0, 3], y=T_camera_paths[:, 1, 3], z=T_camera_paths[:, 2, 3])
fig.add_scatter3d(x=T_keyframes[:, 0, 3], y=T_keyframes[:, 1, 3], z=T_keyframes[:, 2, 3])
fig.add_scatter3d(x=T_camera_paths_new[:, 0, 3], y=T_camera_paths_new[:, 1, 3], z=T_camera_paths_new[:, 2, 3])
fig.update_layout(scene_aspectmode='data')
fig.show()

Now export

In [ ]:
print(data['camera_path'][0])
def create_cam_path(T):
    # return {'camera_to_world': T.flatten().tolist(), 'fov': 50, 'aspect': 16 / 9}
    return {'camera_to_world': T.flatten().tolist(), 'fov': 50, 'aspect': 9 / 16}
data['render_height'] = 800
data['render_width'] = 450

data['camera_path'] = [create_cam_path(T) for T in T_camera_paths_new]
json.dump(data, open('circular_camera_path_3fps.json', 'w'), indent=2)

In [ ]:
# Also create a version that's at the same camera angles etc. that we expect of the real camera
scale_factor, z_height = 6.839369428962543 * 2, -0.09478698496603671
fx, fy, cx, cy = 4760.20112255793, 4754.1622443218685, 348.0, 260.0
FOV = max(np.rad2deg(2 * np.arctan(cx / fx)), np.rad2deg(2 * np.arctan(cy / fy)))
R, height = scale_factor, z_height * scale_factor
t = np.linspace(0, np.pi*2, 24, endpoint=False)[:, None]
positions = np.hstack((R * np.cos(t), R * np.sin(t), height*np.ones_like(t)))
T_camera_paths_new = np.array([pos_lookat_to_matrix(pos, look_at) for pos in positions])
def create_cam_path(T):
    # return {'camera_to_world': T.flatten().tolist(), 'fov': 50, 'aspect': 16 / 9}
    return {'camera_to_world': T.flatten().tolist(), 'fov': FOV, 'aspect': cx / cy}
data['render_height'] = cy * 2
data['render_width'] = cx * 2
data['fps'] = 24 / 4  # 6 fps
data['camera_path'] = [create_cam_path(T) for T in T_camera_paths_new]
json.dump(data, open('circular_camera_path_cameras.json', 'w'), indent=2)

So basically, keyframes is the keyframes that we specified, and then `camera_path` is the full interpolated camera poses.

We probably want to be able to do this interpolation ourself too.

If we look at `curve.js:get_curve_object_from_cameras`, we can see that it's using a Catmull-Rom spline to interpolate the camera poses.

In [ ]:
# Modified from wikipedia


QUADRUPLE_SIZE: int = 4


def num_segments(point_chain: tuple) -> int:
    # There is 1 segment per 4 points, so we must subtract 3 from the number of points  
    return len(point_chain) - (QUADRUPLE_SIZE - 1)


def catmull_rom_spline3(
    P0: tuple,
    P1: tuple,
    P2: tuple,
    P3: tuple,
    num_points: int,
    alpha: float = 0.5,
):
    """
    Compute the points in the spline segment
    :param P0, P1, P2, and P3: The (x,y) point pairs that define the Catmull-Rom spline
    :param num_points: The number of points to include in the resulting curve segment
    :param alpha: 0.5 for the centripetal spline, 0.0 for the uniform spline, 1.0 for the chordal spline.
    :return: The points
    """

    # Calculate t0 to t4. Then only calculate points between P1 and P2.
    # Reshape linspace so that we can multiply by the points P0 to P3
    # and get a point for each value of t.
    def tj(ti: float, pi: tuple, pj: tuple) -> float:
        xi, yi = pi
        xj, yj = pj
        dx, dy = xj - xi, yj - yi
        l = (dx ** 2 + dy ** 2) ** 0.5
        return ti + l ** alpha

    t0: float = 0.0
    t1: float = tj(t0, P0, P1)
    t2: float = tj(t1, P1, P2)
    t3: float = tj(t2, P2, P3)
    t = numpy.linspace(t1, t2, num_points).reshape(num_points, 1)

    A1 = (t1 - t) / (t1 - t0) * P0 + (t - t0) / (t1 - t0) * P1
    A2 = (t2 - t) / (t2 - t1) * P1 + (t - t1) / (t2 - t1) * P2
    A3 = (t3 - t) / (t3 - t2) * P2 + (t - t2) / (t3 - t2) * P3
    B1 = (t2 - t) / (t2 - t0) * A1 + (t - t0) / (t2 - t0) * A2
    B2 = (t3 - t) / (t3 - t1) * A2 + (t - t1) / (t3 - t1) * A3
    points = (t2 - t) / (t2 - t1) * B1 + (t - t1) / (t2 - t1) * B2
    return points


def catmull_rom_chain(points: tuple, num_points: int) -> list:
    """
    Calculate Catmull-Rom for a sequence of initial points and return the combined curve.
    :param points: Base points from which the quadruples for the algorithm are taken
    :param num_points: The number of points to include in each curve segment
    :return: The chain of all points (points of all segments)
    """
    point_quadruples = (
        (points[idx_segment_start + d] for d in range(QUADRUPLE_SIZE))
        for idx_segment_start in range(num_segments(points))
    )
    all_splines = (catmull_rom_spline(*q, num_points) for q in point_quadruples)

    chain = [chain_point for spline in all_splines for chain_point in spline]  # flatten
    return chain
catmull_rom_chain(np.arange(15), 100)

In [ ]:
T_interpolated = generate_interpolated_path(T_keyframes, 96, )
fig.add_scatter3d(x=T_interpolated[:, 0, 3], y=T_interpolated[:, 1, 3], z=T_interpolated[:, 2, 3])